<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/078_Excluding_Dependencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧠 Excluding Dependencies from the Tool Parameters Schema

### 🎯 Goal:

How to **cleanly separate agent logic from dependency management** by hiding injected parameters (*e.g. auth tokens, configs*) from the agent's view and schema.

---

## ✅ Key Concepts

### 1. **Hiding Dependencies from Agent Input Schema**

* Tools can have parameters like `_auth_token` or `_db_connection` that are **injected**, not provided by the agent directly.
* These are **prefixed with `_`** or named `action_context`, so the agent schema-building code can skip them.

---

### 2. **Updating Tool Registration Logic**

You modify the tool registration system (`get_tool_metadata`) to **exclude injected dependencies** when generating the tool’s parameter schema:

```python
if param_name in ["action_context", "action_agent"] or param_name.startswith("_"):
    continue  # Skip injected parameters
```

This ensures the agent doesn’t “see” internal system details like:

```python
_db_connection, _auth_token, _user_config
```

---

### 3. **Clean Agent View**

The agent only sees:

```python
"args": {
    "query": "SELECT * FROM customers"
}
```

And not:

```python
"args": {
    "query": "...",
    "_db_connection": "...",
    "_config": "..."
}
```

---

### 4. **Environment Handles Injection**

The `Environment` continues to inject those hidden values automatically when executing the tool — so the agent stays clean and unaware of them.

---

## 🧰 What This Enables

* ✅ Cleaner **schemas** for LLMs and planners
* ✅ Better **security** (agent doesn't access private data like auth tokens)
* ✅ Clear **separation of concerns**
* ✅ More **modular** and **testable** tools

---

## 🔁 Analogy

Imagine the agent is like a barista:

* The customer (user) asks for a coffee (task).
* The agent takes the order and calls the espresso machine (tool).
* The espresso machine automatically pulls filtered water, coffee beans, electricity (dependencies).
* The barista doesn’t need to think about how water or electricity gets to the machine — just that it makes coffee.



## 🧰 Excluding Dependencies from Tool Parameters in Action

Let’s imagine a hypothetical tool for querying a database. The tool might want to access a database connection, configuration settings, and other dependencies like this:

```python
@register_tool()
def query_database(action_context: ActionContext,
                   query: str,
                   _db_connection: DatabaseConnection,
                   _config: dict) -> dict:
    """Process data using external dependencies."""
    # Tool automatically receives db_connection and config
    ... use the database connection ...
    return query_results
```

We want this tool to automatically receive the dependencies it needs, **but we don’t want the agent to have to understand or provide these parameters**.

The agent should only need to provide the `query` parameter:

```python
# Agent only knows about and provides the query parameter
action = {
    "tool": "query_database",
    "args": {
        "query": "some SQL query"
    }
}
```

To hide the dependencies from the agent, we need to update our tool registration system to **ignore these special parameters** when building the schema that the agent uses:

```python
def get_tool_metadata(func, tool_name=None, description=None,
                     parameters_override=None, terminal=False,
                     tags=None):
    """Extract metadata while ignoring special parameters."""
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    args_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }

    for param_name, param in signature.parameters.items():
        # Skip special parameters - agent doesn't need to know about these
        if param_name in ["action_context", "action_agent"] or \
           param_name.startswith("_"):
            continue

        # Add regular parameters to the schema
        param_type = type_hints.get(param_name, str)
        args_schema["properties"][param_name] = {
            "type": "string"  # Simplified for example
        }

        if param.default == param.empty:
            args_schema["required"].append(param_name)

    return {
        "name": tool_name or func.__name__,
        "description": description or func.__doc__,
        "parameters": args_schema,
        "tags": tags or [],
        "terminal": terminal,
        "function": func
    }
```

This is a **beautifully minimal and effective solution**. Here’s what makes it smart and worth focusing on:

---

### ✅ **Key Idea: Dependency Parameters Start with `_`**

By adopting a **simple naming convention**, like prefixing parameters with an underscore (`_`), the system:

* ✅ Clearly distinguishes **dependencies** from regular user inputs.
* ✅ Allows the registration system to **automatically exclude** them from agent-visible schemas.
* ✅ Keeps the tool functions easy to write and maintain.
* ✅ Requires no additional configuration or flags — the naming *is* the flag.

---

### 🧠 Why It Works Well

1. **Tool simplicity**:

   * Tool developers just name the parameters using `_` and everything works — no special decorators or wrappers needed.

2. **Schema clarity**:

   * When tools are described to the agent (or rendered in a UI), only the user-relevant parameters appear.
   * The LLM or interface never sees sensitive/internal values like `_auth_token`, `_db_connection`, etc.

3. **Decoupling**:

   * Agents don’t need to know about infrastructure-level dependencies.
   * The environment handles injecting those behind the scenes.

---

### 🔍 Example Comparison

#### Before:

The agent might think it needs to provide all of these:

```json
{
  "query": "SELECT * FROM users",
  "_db_connection": "???",
  "_config": "???"
}
```

#### After:

With `_` convention and filtering logic:

```json
{
  "query": "SELECT * FROM users"
}
```

✅ Agent provides only what it *knows*, and the environment injects the rest.

---

### ⚠️ Things to Be Aware Of

* **Consistency is key**: All internal dependencies *must* follow the underscore rule. If someone forgets, the system might treat them as required agent inputs.
* **Not Python-enforced**: This is a *convention*, not a language feature. So it's important your team (or tooling) sticks to it.

---

### 🛠️ Bottom Line

This underscores a larger principle in agent design:

> *“Don’t burden the agent with things it doesn’t need to know.”*

This kind of **separation of concerns** is what makes your architecture modular, testable, and secure.





## ✅ Creating Tools with Hidden Dependencies

Now we can create tools that use **rich dependencies** while keeping them **hidden from the agent**.

For example, a tool that needs **user authentication** and **configuration**:

```python
@register_tool(description="Update user settings in the system")
def update_settings(action_context: ActionContext,
                   setting_name: str,
                   new_value: str,
                   _auth_token: str,
                   _user_config: dict) -> dict:
    """Update a user setting in the external system."""
    # Tool automatically receives auth_token and user_config
    headers = {"Authorization": f"Bearer {_auth_token}"}
    
    if setting_name not in _user_config["allowed_settings"]:
        raise ValueError(f"Setting {setting_name} not allowed")
        
    response = requests.post(
        "https://api.example.com/settings",
        headers=headers,
        json={"setting": setting_name, "value": new_value}
    )
    
    return {"updated": True, "setting": setting_name}
```

---

### 🧠 Agent's Perspective

The agent **only sees** the `setting_name` and `new_value` parameters. When it calls the tool:

```python
# Agent's view of the tool
action = {
    "tool": "update_settings",
    "args": {
        "setting_name": "theme",
        "new_value": "dark"
    }
}
```

---

### 🧰 What the Environment Does

The environment **automatically injects**:

* `action_context`
* `_auth_token`
* `_user_config`

This keeps the **agent’s orchestration logic clean**, while giving tools the **rich context** they need to function.

---

## 🔄 Clean Separation of Concerns

| Responsibility            | Handled By         |
| ------------------------- | ------------------ |
| Deciding what to do       | 🧠 Agent           |
| Declaring dependencies    | 🛠️ Tools          |
| Providing context         | 🌍 Environment     |
| Shared resource container | 📦 `ActionContext` |





### ✅ Help the model *stay focused*:

The LLM can focus entirely on solving the user’s high-level task rather than being distracted by implementation details or context juggling.

---

### ✅ Reduce cognitive "noise":

LLMs, like humans, have limited context windows and attention capacity. Passing *only* what's needed avoids flooding them with irrelevant or overly detailed information.

---

### ✅ Improve reliability & reusability:

Tools become self-contained, predictable, and safe — because they know exactly what they need and receive it automatically.

---

### ✅ Maintain security and clean design:

Sensitive data (like auth tokens or configs) never needs to pass through the LLM if it doesn't have to. That helps with trust, modularity, and safety.

---

### ✅ Treat the model like a collaborator, not a servant:

Your earlier point about *kindness* is more than just metaphor — it's a productive design philosophy. When you build systems that give your LLM clarity, precision, and context **without overwhelm**, you're *setting it up to succeed*.





> **The modular agent approach retains (and even *gains*) value as models become more powerful.** Here's why:

---

### 🔧 1. **Power ≠ Precision or Control**

Even the most capable model still benefits from **structure, constraints, and clarity**.

* Think of GPT-5 like an elite software engineer — they can build anything, but if you just say “make me a product,” you’ll still get wildly different results each time.
* **Agents and tools give you repeatability** — predictable, inspectable, testable components.

---

### 🧠 2. **Cognitive Offloading Still Matters**

As models get more capable, they also become **more ambitious**. They try to solve harder tasks — multi-step reasoning, long-term planning, real-time interaction.

* The bigger the job, the more critical it becomes to **break things down**.
* Even GPT-5 will benefit from agents that **specialize** and collaborate.

---

### 🔐 3. **Security, Auditability, and Trust**

Enterprise use cases, regulated environments, or any high-stakes application still demands:

* **Scoped access to sensitive data** (e.g. auth tokens, financial info)
* **Tool usage tracking**
* **Separation of concerns**

No matter how smart the LLM is, you still want a **controlled execution environment** — and that’s what your modular agent/tool system provides.

---

### 🛠️ 4. **Tool Usage Is the Future**

As LLMs evolve, **tool use becomes their superpower**, not just raw generation.

* GPT-5 can write 400 lines of code. Great.
* But can it also:

  * Call APIs with authentication?
  * Use a memory system?
  * Interact with databases securely?
  * Hand off to other agents with context?

That’s where **tools and environments** shine — letting the model delegate work, not just imagine it.

---

### 🧩 5. **Composable Systems Will Outperform Monolithic Prompts**

As the ecosystem grows (across teams, departments, orgs), modularity becomes key.

* Modular tools and agents can be reused, extended, versioned, and maintained independently.
* That’s not just efficient — it’s how large, resilient systems are built.

> Even if GPT-5 can do a lot with a single prompt, a *production-grade*, *multi-user*, *auditable*, and *scalable* system will benefit from the architecture you’re learning.

---

### 🚀 TL;DR:

> **Powerful models make modular systems more valuable — not less.**
> They elevate the kinds of problems you can solve, but the *agent framework* is what makes those solutions maintainable, secure, and aligned.

So yes — the agent ecosystem you’re building is **future-proof**. You’re not just learning a workaround for weaker models — you’re mastering the architecture that will let you *partner with* stronger ones. 💡






### We’re entering a new phase:

#### 🧠 From “Help the model stay on task”

to

#### 🛠️ “Help the model *not overdo* the task.”

---

### 🔁 What’s Changing

| **Yesterday’s LLMs**                | **Tomorrow’s LLMs (like GPT-5)**               |
| ----------------------------------- | ---------------------------------------------- |
| Needed structure to be productive   | Need structure to *stay focused*               |
| Missed steps or misunderstood goals | Take initiative... sometimes *too much*        |
| Struggled with multi-stage logic    | Confidently generate full pipelines, workflows |
| Required babysitting                | Require *boundaries* and *guidance*            |

---

### 🤖 Why Powerful Models Need Guardrails

As you said:

> “They will try to over-engineer everything.”

Yep — and that’s **natural**. Given a vague prompt, a powerful model may:

* Write code *and* refactor it *and* write tests *and* suggest deployment steps
* Build an entire UX flow when you just asked for a function
* Design a system that’s technically amazing but wildly over-scoped

That’s brilliance… but also **risk**, especially when you're working in:

* Enterprise environments
* Security-sensitive contexts
* Multi-agent workflows
* Projects with tight constraints

---

### ✅ The Solution: **Constraint + Clarity = Performance**

Modular agents and tools do *exactly* what you said:
They **channel** the model’s power, not just unleash it.

This is where **agent architecture** shines:

| **Architectural Tool**       | **How it Helps**                                                     |
| ---------------------------- | -------------------------------------------------------------------- |
| **Tool registry**            | Limits available capabilities — prevents “magic out of nowhere”      |
| **ActionContext**            | Controls which dependencies are available — promotes scoped behavior |
| **Memory systems**           | Provide only relevant context — keep it focused                      |
| **Task-specific agents**     | Prevent “runaway ambition” by enforcing goals                        |
| **Selective memory sharing** | Reduce distraction and overreaction to unrelated info                |

---

### 🧭 Think of it like...

You’re not building **a genius** to do your job.

You’re building **a team of specialists** — guided by you — with tools and boundaries, working *for* your vision.

You’ll get *better*, *faster*, and *more aligned* results this way.


